In [1]:
! pip install openpyxl
! pip install --upgrade deep_translator
! pip install msoffcrypto-tool

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Processing - PEFR_asthma_114_medinfo_07.15.xlsx
import pandas as pd
import io
import msoffcrypto
from deep_translator import MyMemoryTranslator


def decrypt_excel_file(file_path, sheet_name, password):
    # Decrypt the file
    decrypted_workbook = io.BytesIO()
    with open(file_path, "rb") as file:
        office_file = msoffcrypto.OfficeFile(file)
        office_file.load_key(password=password)
        office_file.decrypt(decrypted_workbook)

    # Read the decrypted file into a pandas DataFrame
    df = pd.read_excel(decrypted_workbook, sheet_name=sheet_name)
    return df


# Function to translate text, handling empty/null values
def translate_text(text):
    if pd.isna(text) or text == "":
        return "No Response"
    try:
        return MyMemoryTranslator(source="ko-KR", target="en-US").translate(str(text))
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # Return original text if translation fails


def process_excel_file(file_path, password, sheet_name, output_file):
    df = decrypt_excel_file(file_path, sheet_name, password)

    # Drop specified columns
    df = df.drop(columns=["BCODE", "UID1", "UID2"])

    # # Translate 'occupation' and 'address' columns
    print("Translating occupation column...")
    df["occupation_en"] = df["occupation"].apply(translate_text)
    print("Translating address column...")
    df["address_en"] = df["address"].apply(translate_text)

    # Display the first few rows of the processed dataframe
    print(df.head())

    # Save the processed dataframe to a new CSV file
    df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print(
        "Processing complete. Data saved to '2024-Data-Cleaned/processed_data_asthma_patient_medinfo.csv'"
    )

    print("Medical Info Data")

    display(df)

    return df


# Path to your password-protected Excel file
file_path = "2024-Data/PEFR_asthma_114_medinfo_07.15.xlsx"
password = "1234"
sheet_name = "PEFR_114명"
output_file = "2024-Data-Cleaned/processed_data_asthma_patient_medinfo.csv"

patient_medical_info_df = process_excel_file(
    file_path, password, sheet_name, output_file
)

Translating occupation column...
Translating address column...
       ID  Age Sex Smoke  Smoke_amount  Height  Weight        BMI       BSA  \
0  SB-001   43   M    NS             0     156      76  31.229454  1.814754   
1  SB-002   66   M    NS             0     163      58  21.829952  1.620528   
2  SB-003   61   M    ES            20     173      67  22.386314  1.794358   
3  SB-004   49   M    NS             0     142      51  25.292601  1.418332   
4  SB-005   53   M    NS             0     156      70  28.763971  1.741647   

  occupation                         address  occupation_en  \
0         주부            경기도 시흥시 마유로 443번길 1     : HOUSEWIFE   
1        제조업   경기 부천시 원미구 중동 무지개마을 1201-1602  Manufacturing   
2         타일                 인천시 부평구 수변로 333           Tile   
3         주부                인천 서구 연희동 799-14    : HOUSEWIFE   
4        무응답  경기 부천시 원미구 상3동 라일락마을 2333-1503      N/A&nbsp;   

                                          address_en  
0      1, Mayu-ro 443beon-gi

,ID,Age,Sex,Smoke,Smoke_amount,Height,Weight,BMI,BSA,occupation,address,occupation_en,address_en
0,SB-001,43,M,NS,0,156,76,31.229454,1.814754,주부,경기도 시흥시 마유로 443번길 1,: HOUSEWIFE,"1, Mayu-ro 443beon-gil, Siheung-si, Gyeonggi-"
1,SB-002,66,M,NS,0,163,58,21.829952,1.620528,제조업,경기 부천시 원미구 중동 무지개마을 1201-1602,Manufacturing,"1201-1602, Rainbow Village, Jungdong, Wonmi-gu..."
2,SB-003,61,M,ES,20,173,67,22.386314,1.794358,타일,인천시 부평구 수변로 333,Tile,"333, Subyun-ro, Bupyeong-gu, Incheon"
3,SB-004,49,M,NS,0,142,51,25.292601,1.418332,주부,인천 서구 연희동 799-14,: HOUSEWIFE,"799-14 Yeonhui-dong, Seo-gu, Incheon"
4,SB-005,53,M,NS,0,156,70,28.763971,1.741647,무응답,경기 부천시 원미구 상3동 라일락마을 2333-1503,N/A&nbsp;,"Lilac Village 2333-1503, Sang3-dong, Wonmi-gu,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,SB-135,79,M,NS,0,163,67,25.217359,1.741726,무직,부천시 원미구 도당동 137-4,Not Employed,"137-4, Dodang-dong, Wonmi-gu, Bucheon-si"
110,SB-137,76,F,NS,0,160,66,25.781250,1.712698,주부,경기도 부천시 부흥로 71,: HOUSEWIFE,"71, Bupheung-ro, Bucheon-si, G"
111,SB-139,50,F,ES,20,162,64,24.386526,1.697056,건설업,부천시 원미구 중2동 그린타운 한신A,Construction,"Hanshin A, Green Town, Jung 2-dong, Wonmi-gu, ..."
112,SB-140,69,F,ES,15,170,72,24.913495,1.843909,건설업,부천시 원미구 중3동 중흥마을,Construction,"Jungheung Village, Jung 3-dong, Wonmi-gu, Buch..."


In [10]:
import os
import pandas as pd
import numpy as np

# from datetime import datetime
pd.set_option("future.no_silent_downcasting", True)


def parse_date_column(date_str):
    for date_format in ["%Y-%m-%d", "%d-%m-%Y", "%m/%d/%Y", "%Y/%m/%d"]:
        try:
            return pd.to_datetime(date_str, format=date_format).date()
        except ValueError:
            logger.warning(f"Unable to parse date: {date_str}")
            continue
    return None


import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def clean_data(df, filename, patient_id):

    logger.info(f"Processing file: {filename}")

    # Find the index of the row where the third column (original G) is "Morning"
    morning_row_index = df.index[df.iloc[:, 1] == "Morning"].tolist()
    if morning_row_index:

        # If "Morning" is found, set that row as header and remove it from the data
        header_row = morning_row_index[0]
        df.columns = df.iloc[header_row]
        df = df.drop(df.index[: header_row + 1]).reset_index(drop=True)

    else:
        print(f"Warning: 'Morning' not found in column G for file {filename}")

        # If "Morning" is not found, use default column names
        df.columns = ["B", "G", "H", "I"]

    # Replace empty cells or whitespace-only cells with NaN
    df = df.replace(r"^\s*$", np.nan, regex=True)

    # Drop rows where all cells are NaN
    df = df.dropna(how="all")
    df.columns = ["Date", "Morning", "Afternoon", "Other"]

    # Parse dates
    df["Date"] = df["Date"].apply(parse_date_column)

    # Remove rows where date parsing failed
    df = df.dropna(subset=["Date"])

    # Convert 'Date' column to datetime
    df["Date"] = pd.to_datetime(df["Date"])

    # Convert 'Morning', 'Afternoon', and 'Other' columns to numeric, coercing errors to NaN
    for col in ["Morning", "Afternoon", "Other"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Handle 'Other' column
    mask = df["Other"].notna()

    logger.info(f"Original data shape: {df.shape}")
    df.to_csv(f"2024-Data-Cleaned/Original_Data/{patient_id}-OG.csv", index=False)

    # Move 'Other' to 'Morning' if 'Morning' is empty
    morning_mask = mask & df["Morning"].isna()
    df.loc[morning_mask, "Morning"] = df.loc[morning_mask, "Other"]
    df.loc[morning_mask, "Other"] = np.nan

    # Move 'Other' to 'Afternoon' if 'Afternoon' is empty and 'Other' still has value
    mask = df["Other"].notna()  # Recalculate mask
    afternoon_mask = mask & df["Afternoon"].isna()
    df.loc[afternoon_mask, "Afternoon"] = df.loc[afternoon_mask, "Other"]
    df.loc[afternoon_mask, "Other"] = np.nan

    if df.empty:
        logger.warning(f"Cleaned dataframe is empty for file {filename}")
        return None

    logger.info(f"Cleaned data shape: {df.shape}")
    df.to_csv(f"2024-Data-Cleaned/Cleaned_Data/{patient_id}-Clean.csv", index=False)

    return df


def load_and_clean_data(folder_path):
    patient_data_dict = {}

    # Create directory if it doesn't exist
    Original_Data_dir = "2024-Data-Cleaned/Original_Data"
    os.makedirs(Original_Data_dir, exist_ok=True)
    Cleaned_Data_dir = "2024-Data-Cleaned/Cleaned_Data"
    os.makedirs(Cleaned_Data_dir, exist_ok=True)

    for filename in os.listdir(folder_path):
        patient_id = filename.split(" ")[0]
        # Read the file without setting a header initially
        df = pd.read_excel(
            os.path.join(folder_path, filename),
            sheet_name="Database",
            engine="openpyxl",
            header=None,
            usecols=[1, 6, 7, 8],
        )  # 0-based index for columns B, G, H, I
        patient_data_dict[patient_id] = clean_data(df, filename, patient_id)

    return patient_data_dict


cleaned_patient_data_dict = load_and_clean_data(folder_path="2024-Data/SCH_asthma_114/")

INFO:__main__:Processing file: SB-001 (2020-02-05).xlsx
INFO:__main__:Original data shape: (1336, 4)
INFO:__main__:Cleaned data shape: (1336, 4)
INFO:__main__:Processing file: SB-002 (2017-01-29).xlsx
INFO:__main__:Original data shape: (572, 4)
INFO:__main__:Cleaned data shape: (572, 4)
INFO:__main__:Processing file: SB-003 (2020-03-29).xlsx
INFO:__main__:Original data shape: (1064, 4)
INFO:__main__:Cleaned data shape: (1064, 4)
INFO:__main__:Processing file: SB-004 (2021-09-04).xlsx
INFO:__main__:Original data shape: (1823, 4)
INFO:__main__:Cleaned data shape: (1823, 4)
INFO:__main__:Processing file: SB-005 (2017-03-11).xlsx
INFO:__main__:Original data shape: (281, 4)
INFO:__main__:Cleaned data shape: (281, 4)
INFO:__main__:Processing file: SB-006 (2020-01-08).xlsx
INFO:__main__:Original data shape: (1327, 4)
INFO:__main__:Cleaned data shape: (1327, 4)
INFO:__main__:Processing file: SB-007 (2017-05-15).xlsx
INFO:__main__:Original data shape: (309, 4)
INFO:__main__:Cleaned data shape: 

In [12]:
import pandas as pd


def impute_time_series_data(df):
    def get_neighbor_values(df, index, column):
        neighbors = []
        index_list = df.index.tolist()
        current_index_position = index_list.index(index)

        if current_index_position > 0:
            prev_index = index_list[current_index_position - 1]
            neighbors.extend(
                [df.loc[prev_index, "Morning"], df.loc[prev_index, "Afternoon"]]
            )

        if current_index_position < len(index_list) - 1:
            next_index = index_list[current_index_position + 1]
            neighbors.extend(
                [df.loc[next_index, "Morning"], df.loc[next_index, "Afternoon"]]
            )

        if column == "Morning" and pd.notna(df.loc[index, "Afternoon"]):
            neighbors.append(df.loc[index, "Afternoon"])
        elif column == "Afternoon" and pd.notna(df.loc[index, "Morning"]):
            neighbors.append(df.loc[index, "Morning"])

        return [x for x in neighbors if pd.notna(x)]

    for column in ["Morning", "Afternoon"]:
        for index in df.index:
            if pd.isna(df.loc[index, column]):
                neighbor_values = get_neighbor_values(df, index, column)
                if neighbor_values:
                    imputed_value = round(sum(neighbor_values) / len(neighbor_values))
                    df.loc[index, column] = imputed_value

    logger.info(f"Imputed data shape: {df.shape}")
    df.to_csv(f"2024-Data-Cleaned/Imputed_Data/{patient_id}-recent-Imputed.csv", index=False)
    return df


# Apply the imputation to each patient's data
imputed_patient_data_dict = {}

# Create directory if it doesn't exist
imputed_Data_dir = "2024-Data-Cleaned/Imputed_Data"
os.makedirs(imputed_Data_dir, exist_ok=True)

for patient_id, patient_df in cleaned_patient_data_dict.items():
    imputed_patient_data_dict[patient_id] = impute_time_series_data(patient_df)

INFO:__main__:Imputed data shape: (1336, 4)
INFO:__main__:Imputed data shape: (572, 4)
INFO:__main__:Imputed data shape: (1064, 4)
INFO:__main__:Imputed data shape: (1823, 4)
INFO:__main__:Imputed data shape: (281, 4)
INFO:__main__:Imputed data shape: (1327, 4)
INFO:__main__:Imputed data shape: (309, 4)
INFO:__main__:Imputed data shape: (2100, 4)
INFO:__main__:Imputed data shape: (975, 4)
INFO:__main__:Imputed data shape: (266, 4)
INFO:__main__:Imputed data shape: (2093, 4)
INFO:__main__:Imputed data shape: (594, 4)
INFO:__main__:Imputed data shape: (773, 4)
INFO:__main__:Imputed data shape: (1793, 4)
INFO:__main__:Imputed data shape: (640, 4)
INFO:__main__:Imputed data shape: (667, 4)
INFO:__main__:Imputed data shape: (1990, 4)
INFO:__main__:Imputed data shape: (309, 4)
INFO:__main__:Imputed data shape: (648, 4)
INFO:__main__:Imputed data shape: (422, 4)
INFO:__main__:Imputed data shape: (194, 4)
INFO:__main__:Imputed data shape: (1522, 4)
INFO:__main__:Imputed data shape: (1422, 4)
I

In [14]:
import pandas as pd


def create_patient_summary(patient_data_dict, output_file):
    summary_data = []

    for patient_id, patient_data in patient_data_dict.items():
        row_count = len(patient_data)
        null_morning = patient_data["Morning"].isna().sum()
        null_afternoon = patient_data["Afternoon"].isna().sum()
        null_other = patient_data["Other"].isna().sum()

        summary_data.append(
            {
                "Patient_ID": patient_id,
                "Total_Rows": row_count,
                "Null_Morning": null_morning,
                "Null_Afternoon": null_afternoon,
                "Null_Other": null_other,
            }
        )

    summary_df = pd.DataFrame(summary_data)

    # Display the summary dataframe
    print("\nPatient Perf Data Summary:")
    display(summary_df)

    # Optional: Save to CSV
    summary_df.to_csv(output_file, index=False)

    return summary_df


output_file = "2024-Data-Cleaned/cleaned_patient_perf_data_summary.csv"
# Create the summary dataframe
patient_summary = create_patient_summary(cleaned_patient_data_dict, output_file)

output_file = "2024-Data-Cleaned/imputed_patient_perf_data_summary.csv"
# Create the summary dataframe
patient_summary = create_patient_summary(imputed_patient_data_dict, output_file)


Patient Perf Data Summary:


,Patient_ID,Total_Rows,Null_Morning,Null_Afternoon,Null_Other
0,SB-001,1336,0,0,1336
1,SB-002,572,0,0,572
2,SB-003,1064,0,0,1064
3,SB-004,1823,0,0,1823
4,SB-005,281,0,0,281
...,...,...,...,...,...
95,SB-118,93,0,0,93
96,SB-119,624,0,0,624
97,SB-120,199,0,0,199
98,SB-121,979,0,0,973



Patient Perf Data Summary:


,Patient_ID,Total_Rows,Null_Morning,Null_Afternoon,Null_Other
0,SB-001,1336,0,0,1336
1,SB-002,572,0,0,572
2,SB-003,1064,0,0,1064
3,SB-004,1823,0,0,1823
4,SB-005,281,0,0,281
...,...,...,...,...,...
95,SB-118,93,0,0,93
96,SB-119,624,0,0,624
97,SB-120,199,0,0,199
98,SB-121,979,0,0,973


In [16]:
import pandas as pd


def combine_patient_dataframes(patient_data_dict, output_file):
    # Create a list to store all dataframes
    all_dataframes = []

    # Iterate through the dictionary
    for patient_id, df in patient_data_dict.items():
        # Add a new column for patient ID
        df["Patient_ID"] = patient_id

        # Append the dataframe to our list
        all_dataframes.append(df)

    # Concatenate all dataframes in the list
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Reorder columns to have Patient_ID first
    columns_order = ["Patient_ID"] + [
        col for col in combined_df.columns if col != "Patient_ID"
    ]
    combined_df = combined_df[columns_order]

    # Display the first few rows of the combined dataframe
    combined_df.to_csv(output_file, index=False, encoding="utf-8-sig")

    return combined_df


output_file = "2024-Data-Cleaned/combined_cleaned_patient_asthma_data.csv"

# Use the function to create the combined dataframe
combined_patient_data = combine_patient_dataframes(
    cleaned_patient_data_dict, output_file
)

output_file = "2024-Data-Cleaned/combined_imputed_patient_asthma_data.csv"

# Use the function to create the combined dataframe
combined_imputed_patient_data = combine_patient_dataframes(
    imputed_patient_data_dict, output_file
)